---
# **LAB 9 - CUDA in Python**
---

# ▶️ CUDA setup

In [ ]:
!nvcc --version

In [ ]:
!nvidia-smi

[GPU Compute Capability](https://developer.nvidia.com/cuda-gpus)

In [ ]:
!git clone https://github.com/giulianogrossi/GPUcomputing.git

# 🐍 kernel for sum

In [ ]:
from numba import cuda
import numpy as np

# getting device information
cuda.detect();

In [ ]:
device = cuda.get_current_device()
attribs = [s for s in dir(device) if s.isupper()]        # all attribs
for attr in attribs:
  print(attr, '=', getattr(device, attr))

In [ ]:
# CUDA kernels written with `@cuda.jit` do not return values
@cuda.jit
def add_kernel(x, y, z):
  """Perform vector sum z = x * y
  """
    
  # The actual values of the following CUDA-provided variables for thread and block indices,
  # like function parameters, are not known until the kernel is launched.
                          
  # This Numba-provided convenience function is equivalent to
  idx = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x

  # This calculation gives a unique thread index within the entire grid (see the slides above for more)
  # idx = cuda.grid(1)      # 1 = one dimensional thread grid, returns a single value.

  # This thread will do the work on the data element with the same index as its own
  # unique index within the grid.
  z[idx] = x[idx] + y[idx]

Generate data and device setup/transfert...

In [ ]:
# populate data
n = 512*1024*1024
print(f'num bytes (3 arrays) = {n*4*3/1024/1024} MB')
x = np.arange(n).astype(np.int32) # [0...4095] on the host
y = np.ones_like(x)               # [1...1] on the host

d_x = cuda.to_device(x) # Copy of x on the device
d_y = cuda.to_device(y) # Copy of y on the device
d_out = cuda.device_array_like(d_x) # Like np.array_like, but for device arrays

In [ ]:
from timeit import default_timer as timer

# execution of the kernel
start = timer()
w = x + y    # sum
end = timer()
print(f"Elapsed (sec) = {end - start}")

Define the kernel parameters...

In [ ]:
# Because of how we wrote the kernel above, we need to have a 1 thread to one data element mapping,
# therefore we define the number of threads in the grid (128*32) to equal n (4096).
threads_per_block = 1024
blocks_per_grid = int(n/threads_per_block)

launch the kernel...

In [ ]:
# execution of the kernel
start = timer()
add_kernel[blocks_per_grid, threads_per_block](d_x, d_y, d_out)
# host and device sync
cuda.synchronize()
end = timer()
print(f"Elapsed (sec) = {end - start}")

In [ ]:
!nvidia-smi

CUDA object...

In [ ]:
type(d_x)

Remove the reference from the current namespace

In [ ]:
del d_x, d_y, d_out
del x, y, w

How to measure performance of NUMBA

In [ ]:
from numba import jit
import numpy as np
import time

x = np.arange(100).reshape(10, 10)

@jit(nopython=True)
def go_fast(a): # Function is compiled and runs in machine code
  trace = 0.0
  for i in range(a.shape[0]):
    trace += np.tanh(a[i, i])
  return a + trace

# DO NOT REPORT THIS... COMPILATION TIME IS INCLUDED IN THE EXECUTION TIME!
start = time.time()
go_fast(x)
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

# NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE
start = time.time()
go_fast(x)
end = time.time()
print("Elapsed (after compilation) = %s" % (end - start))

# 🐍 Mat multiplication kernel

A tutorial that explains NumPy axes: [Numpy axes](https://www.sharpsightlabs.com/blog/numpy-axes-explained/)

In [ ]:
from numba import cuda
import time
import numpy as np

In [ ]:
@cuda.jit
def matmul(A, B, C):
  """Perform square matrix multiplication of C = A * B."""
  i, j = cuda.grid(2)
  if i < C.shape[0] and j < C.shape[1]:
    tmp = 0.
    for k in range(A.shape[1]):
      tmp += A[i, k] * B[k, j]
    C[i, j] = tmp

# matrix multiplication using the cpu and C-like programming style (** strongly discouraged **)
def matmul_cpu(A, B):
  """
  Perform square matrix multiplication of C = A * B
  """
  C = np.zeros((A.shape[1],B.shape[1]))
  for i in range(A.shape[1]):
    for j in range(B.shape[0]):
      tmp = 0.                            
      for k in range(A.shape[1]):
        tmp += A[i, k] * B[k, j]   # multiply elements in row i of A and column j of B and add to temp
      C[i, j] = tmp    

In [ ]:
# generate random vals
np.random.seed(42)
SIZE = 8000
A = np.ones((SIZE,SIZE)).astype('float32')  # mat 1
B = np.ones((SIZE,SIZE)).astype('float32')  # mat 2
#C = np.zeros((SIZE,SIZE)).astype('float32')                       # mat where we store answer 

d_A = cuda.to_device(A) # Copy of A on the device
d_B = cuda.to_device(B) # Copy of B on the device
d_C = cuda.device_array_like(A) # malloc on the device

# data type
d_A.dtype

Kernel parameters...

In [ ]:
block = (32, 32)  # each block will contain 32x32 threads, typically 128 - 512 threads/block
grid_x = int(np.ceil(A.shape[0] / block[0]))
grid_y = int(np.ceil(A.shape[1] / block[1]))
grid = (grid_x, grid_y)  # we calculate the gridsize (number of blocks) from array
print(grid)
print(f"The kernel will be executed up to element {block[0]*grid_x}")

In [ ]:
# execution of the kernel
start = time.time()
matmul[grid, block](d_A, d_B, d_C)
# host and device sync
cuda.synchronize()
end = time.time()
print("Elapsed (sec) = %s" % (end - start))

In [ ]:
C = d_C.copy_to_host()
print(C)

In [ ]:
%%time
# execution of the python function in C-like programming style (strongly discouraged!)
D = matmul_cpu(A, B)

In [ ]:
# using numpy function
start = time.time()
C = np.dot(A, B)
end = time.time()
print("Elapsed (sec) = %s" % (end - start))

In [ ]:
print(C)

# 🔴 TODO

Improved matrix multiplication usin SMEM

```
# Controls threads per block and shared memory usage.
# The computation will be done on blocks of TPBxTPB elements.
# TPB should not be larger than 32 in this example
TPB = 16
@cuda.jit
def fast_matmul(A, B, C):

  # Define an array in the shared memory

  # Each thread computes one element in the result matrix.

  # Wait until all threads finish preloading

  # Computes partial product on the shared memory

  # Wait until all threads finish computing
```



In [ ]:
# Controls threads per block and shared memory usage.
# The computation will be done on blocks of TPBxTPB elements.
# TPB should not be larger than 32 in this example
TPB = 16

@cuda.jit
def fast_matmul(A, B, C):
    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=np.float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=np.float32)

    x, y = cuda.grid(2)

    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bpg = cuda.gridDim.x    # blocks per grid

    if x >= C.shape[0] and y >= C.shape[1]:
        # Quit if (x, y) is outside of valid C boundary
        return

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0.
    for i in range(bpg):
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * TPB]
        sB[tx, ty] = B[tx + i * TPB, y]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx, j] * sB[j, ty]

        # Wait until all threads finish computing
        cuda.syncthreads()

    C[x, y] = tmp

In [ ]:
# generate random vals
np.random.seed(42)
SIZE = 8000
A = np.ones((SIZE,SIZE)).astype('float32')  # mat 1
B = np.ones((SIZE,SIZE)).astype('float32')  # mat 2
#C = np.zeros((SIZE,SIZE)).astype('float32')                       # mat where we store answer 

d_A = cuda.to_device(A) # Copy of A on the device
d_B = cuda.to_device(B) # Copy of B on the device
d_C = cuda.device_array_like(A) # malloc on the device

# data type
d_A.dtype

Kernel parameters...

In [ ]:
block = (TPB, TPB)  # each block will contain TPBxTPB threads, typically 128 - 512 threads/block
grid_x = int(np.ceil(A.shape[0] / block[0]))
grid_y = int(np.ceil(A.shape[1] / block[1]))
grid = (grid_x, grid_y)  # we calculate the gridsize (number of blocks) from array
print(grid)
print(f"The kernel will be executed up to element {block[0]*grid_x}")

In [ ]:
# execution of the kernel
start = time.time()
fast_matmul[grid, block](d_A, d_B, d_C)

# host and device sync
cuda.synchronize()
end = time.time()
print("Elapsed (sec) = %s" % (end - start))

# 🐍 Histogram

In [ ]:
from numba import cuda
import numpy as np

## 🔴 TODO

In [ ]:
@cuda.jit
def BMP_hist(I, hist):
  '''
  Increment bin counts in hist
    params:
      - I (uint8)  : HxWx3 matrix
      - hist (int) : 256x3 matrix
  '''

  i, j = cuda.grid(2)  
  R = I[i,j,0]
  G = I[i,j,1]
  B = I[i,j,2]
  cuda.atomic.add(hist[0], R, 1)
  cuda.atomic.add(hist[1], G, 1)
  cuda.atomic.add(hist[2], B, 1)

## Gestione immagine e kernel...

In [ ]:
from PIL import Image

#image.save('beach1.bmp')
img = Image.open('GPUcomputing/images/dog.bmp')
img_rgb = img.convert('RGB')
img_rgb

In [ ]:
# converto to numpy (host)
I = np.asarray(img)
print(f"Image size W x H x ch = {I.shape}")

# device data setup
d_I = cuda.to_device(I)
H = np.zeros((3,256)).astype(np.float32)
d_H = cuda.to_device(H)

Kernel parameters...

In [ ]:
block = (16, 16)  # each block will contain 16x16 threads, typically 128 - 512 threads/block
grid_x = int(np.ceil(I.shape[0] / block[0]))
grid_y = int(np.ceil(I.shape[1] / block[1]))
grid = (grid_x, grid_y)  # we calculate the gridsize (number of blocks) from array
print(grid)
print(f"The kernel will be executed up to element {block[0]*grid_x}")

In [ ]:
# kernel launch

BMP_hist[grid, block](d_I, d_H)
hist = d_H.copy_to_host()
print(hist.shape)

In [ ]:
import matplotlib.pyplot as plt

plt.bar(np.arange(256),hist[0])
plt.title('Histogram (R)')
plt.show()
plt.bar(np.arange(256),hist[1])
plt.title('Histogram (G)')
plt.show()
plt.bar(np.arange(256),hist[2])
plt.title('Histogram (B)')
plt.show()

# 🔴 TODO

Calcolare il prodotto di matrici MQDB con kernel CUDA in Python

In [ ]:
from numba import cuda
import numpy as np
import time

def MQDB_gen(n, k, values=None):
  """
  Generate a random MQDB of size n with k (random sized) blocks
    params:
      n (int)                : mat size
      k (int or list)        : the number of blocks or blocks dims (list)
      values (string or num) : the string 'rand' for random values, 
                               a scalar for constant values everywere,
                               'None' for no assignement        
  """
  # blocks dims
  if isinstance(k, (list, np.ndarray)):
    blkdims = k  # list of already defined dims
  else:          # generate new dims
    bins = np.concatenate((np.ones(n), np.zeros(n*(k-1))))
    np.random.shuffle(bins)
    blkdims = np.sum(bins.reshape(k,n),axis=1).astype('int')
  print(f"Mat size n = {n}  --  num blocks k = {blkdims.shape[0]}")
  print('Blocks dims = ', blkdims)
  
  A = np.zeros((n,n))
  # filling of blocks 
  if values is not None:
    end = 0
    for b in blkdims:
      start = end
      end = end + b
      if values == 'rand':
        A[start:end,start:end] = np.random.rand(b,b)
      else:
        A[start:end,start:end] = values*np.ones(b,b)
      
  # return final data
  return A, blkdims

@cuda.jit
def matmul_MQDB(A, B, C, blkdim, blksum):
  """
  Perform MQDB matrix multiplication C = A * B
  params:
      A,B,C (MQDB) : omogeneous MQDB types 
      blkdim (int) : the current block dim
      blksum (int) : sum of the previous blocks sizes    
  """
  row, col = cuda.grid(2)
  if row < blkdim and col < blkdim:
    tmp = 0.
    for k in range(blkdim):
      tmp += A[row+blksum, k+blksum] * B[k+blksum, col+blksum]
    C[row+blksum, col+blksum] = tmp


In [ ]:
# generate random blocks and data
n = 10000
k = 4
A, blkdims = MQDB_gen(n, k, values=1)
B, _ = MQDB_gen(n, blkdims, values=1)
C, _ = MQDB_gen(n, blkdims)

# device mem
d_A = cuda.to_device(A) # Copy of A on the device
d_B = cuda.to_device(B) # Copy of B on the device
d_C = cuda.device_array_like(A) # malloc on the device

In [ ]:
# run the kernels

# warm up 
matmul_MQDB[grid, block](d_A, d_B, d_C, blkdims[0], 0)

start = time.time()
block = (16, 16) # block size
for i in range(k):
  grid_x = int(np.ceil(blkdims[i]/block[0]))
  grid_y = int(np.ceil(blkdims[i]/block[1]))
  grid = (grid_x, grid_y)  
  sum_blks = 0 if i==0 else np.sum(blkdims[0:i])
  # kernel launch
  matmul_MQDB[grid, block](d_A, d_B, d_C, blkdims[i], sum_blks)

# host and device sync
cuda.synchronize()
end = time.time()
print("Elapsed (sec) = %s" % (end - start))


In [ ]:
# print results
print('results: matrx C')
import sys
C = d_C.copy_to_host() # copy to host from device
print(C)

In [ ]:
# using numpy function

start = time.time()
np.dot(A,B)
end = time.time()
print("Elapsed (sec) = %s" % (end - start))

# 🐍 Pseudo-random generator

In [ ]:
from __future__ import print_function, absolute_import

from numba import cuda
from numba.cuda.random import create_xoroshiro128p_states, xoroshiro128p_uniform_float32
import numpy as np

@cuda.jit
def compute_pi(rng_states, iterations, out):
    """Find the maximum value in values and store in result[0]"""
    thread_id = cuda.grid(1)

    # Compute pi by drawing random (x, y) points and finding what
    # fraction lie inside a unit circle
    inside = 0
    for i in range(iterations):
        x = xoroshiro128p_uniform_float32(rng_states, thread_id)
        y = xoroshiro128p_uniform_float32(rng_states, thread_id)
        if x**2 + y**2 <= 1.0:
            inside += 1

    out[thread_id] = 4.0 * inside / iterations

####### run ######
threads_per_block = 64
blocks = 24
rng_states = create_xoroshiro128p_states(threads_per_block * blocks, seed=1)
out = np.zeros(threads_per_block * blocks, dtype=np.float32)
compute_pi[blocks, threads_per_block](rng_states, 10000, out)
print('pi:', out.mean())

In [ ]:
# Create a new stream
stream = cuda.stream()
# Create a pinned array on the host for async transfers
a = cuda.pinned_array(n, dtype=np.int32)
# Create an array with a "default stream“
d_a = cuda.device_array_like(a, stream)
# Numba automatically uses async transfers when streams involvedd_a.copy_to_device(a, stream=stream)
# Kernel launch on stream
kernel[nblocks, nthreads, stream]

# 🐍 Fractals

In [ ]:
from __future__ import print_function, division, absolute_import

from timeit import default_timer as timer
from matplotlib.pylab import imshow, show
import numpy as np

from numba import jit

@jit
def mandel(x, y, max_iters):
    """
    Given the real and imaginary parts of a complex number,
    determine if it is a candidate for membership in the Mandelbrot
    set given a fixed number of iterations.
    """
    i = 0
    c = complex(x, y)
    z = 0.0j
    for i in range(max_iters):
        z = z * z + c
        if (z.real * z.real + z.imag * z.imag) >= 4:
            return i

    return 255


@jit
def create_fractal(min_x, max_x, min_y, max_y, image, iters):
    height = image.shape[0]
    width = image.shape[1]

    pixel_size_x = (max_x - min_x) / width
    pixel_size_y = (max_y - min_y) / height
    for x in range(width):
        real = min_x + x * pixel_size_x
        for y in range(height):
            imag = min_y + y * pixel_size_y
            color = mandel(real, imag, iters)
            image[y, x] = color

    return image

###### run MANDEL-CPU code #########

width = 15000
height = 10000
image = np.zeros((height, width), dtype=np.uint8)

s = timer()
create_fractal(-2.0, 1.0, -1.0, 1.0, image, 20)
e = timer()
print("Execution time: %f seconds" % (e - s))

imshow(image)
show()

In [ ]:
from __future__ import print_function, division, absolute_import

from timeit import default_timer as timer
from matplotlib.pylab import imshow, show
import numpy as np

from numba import cuda


@cuda.jit(device=True)
def mandel(x, y, max_iters):
  """
  Given the real and imaginary parts of a complex number,
  determine if it is a candidate for membership in the Mandelbrot
  set given a fixed number of iterations.
  """
  i = 0
  c = complex(x, y)
  z = 0.0j
  for i in range(max_iters):
    z = z * z + c
    if (z.real * z.real + z.imag * z.imag) >= 4:
      return i

  return 255


@cuda.jit
def create_fractal(min_x, max_x, min_y, max_y, image, iters):
  height = image.shape[0]
  width = image.shape[1]

  pixel_size_x = (max_x - min_x) / width
  pixel_size_y = (max_y - min_y) / height

  x, y = cuda.grid(2)

  if x < width and y < height:
    real = min_x + x * pixel_size_x
    imag = min_y + y * pixel_size_y
    color = mandel(real, imag, iters)
    image[y, x] = color


###### run MENDEL-GPU code #########

width = 15000
height = 10000
image = np.zeros((height, width), dtype=np.uint8)

pixels = width * height
nthreads = 32
nblocksy = (height // nthreads) + 1
nblocksx = (width // nthreads) + 1
s = timer()

create_fractal[(nblocksx, nblocksy), (nthreads, nthreads)](-2.0, 1.0, -1.0, 1.0, image, 20)
cuda.synchronize()
e = timer()
print("Execution time: %f seconds" % (e - s))

imshow(image)
show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import newaxis

def compute_mandelbrot(N_max, some_threshold, nx, ny):
    # A grid of c-values
    x = np.linspace(-2, 1, nx)
    y = np.linspace(-1.5, 1.5, ny)

    c = x[:,newaxis] + 1j*y[newaxis,:]

    # Mandelbrot iteration

    z = c

    # The code below overflows in many regions of the x-y grid, suppress
    # warnings temporarily
    with np.warnings.catch_warnings():
        np.warnings.simplefilter("ignore")
        for j in range(N_max):
            z = z**2 + c
        mandelbrot_set = (abs(z) < some_threshold)

    return mandelbrot_set

mandelbrot_set = compute_mandelbrot(1000, 1., 1000,1000)

plt.imshow(mandelbrot_set.T)

plt.show()

In [ ]:
### Numpy and CPU
s = time.time()
x_cpu *= 5
e = time.time()
print(f"CPU: {e-s}")

### CuPy and GPU
s = time.time()
x_gpu *= 5
cp.cuda.Stream.null.synchronize()
e = time.time()
print(f"GPU: {e-s}")

# ✅ CuPy

Basics of CuPy
In this section, you will learn about the following things:

- Basics of cupy.ndarray

- The concept of current device

- Host-device and device-device array transfer



CuPy is a GPU array backend that implements a subset of NumPy interface. In the following code, `cp` is an abbreviation of `cupy`, following the convention of abbreviating `numpy` to `np`:

In [ ]:
import numpy as np
import cupy as cp
import time

The `cupy.ndarray` class is in its core, which is a compatible GPU alternative of `numpy.ndarray`.

In [ ]:
x_gpu = cp.array([1, 2, 3])

`x_gpu` in the above example is an instance of `cupy.ndarray`. You can see its creation of identical to NumPy’s one, except that `numpy` is replaced with `cupy`. The main difference of `cupy.ndarray` from `numpy.ndarray` is that the content is allocated on the device memory. Its data is allocated on the current device, which will be explained later.

Most of the array manipulations are also done in the way similar to NumPy. Take the Euclidean norm (a.k.a L2 norm) for example. NumPy has `numpy.linalg.norm()` to calculate it on CPU.

In [ ]:
x_cpu = np.array([1, 2, 3])
l2_cpu = np.linalg.norm(x_cpu)
print(l2_cpu)

x_gpu = cp.array([1, 2, 3])
l2_gpu = cp.linalg.norm(x_gpu)
print(l2_gpu)

CuPy has a concept of current devices, which is the default device on which the allocation, manipulation, calculation, etc., of arrays are taken place. Suppose the ID of current device is 0. The following code allocates array contents on GPU 0.

In [ ]:
x_on_gpu0 = cp.array([1, 2, 3, 4, 5])
# cp.cuda.Device(1).use() # trigger dev 1

__Move arrays to a device__ 

`cupy.asarray()` can be used to move a `numpy.ndarray`, a `list`, or any `object` that can be passed to `numpy.array()` to the current device:

In [ ]:
x_cpu = np.array([1, 2, 3])
x_gpu = cp.asarray(x_cpu)  # move the data to the current device.

`cupy.asarray()` can accept `cupy.ndarray`, which means we can transfer the array between devices with this function.

__Move array from a device to the host__

Moving a device array to the host can be done by `cupy.asnumpy()` as follows:

In [ ]:
x_gpu = cp.array([1, 2, 3])  # create an array in the current device
x_cpu = cp.asnumpy(x_gpu)  # move the array to the host.

# We can also use cupy.ndarray.get():
x_cpu = x_gpu.get()

__How to write CPU/GPU agnostic code__

The compatibility of CuPy with NumPy enables us to write CPU/GPU generic code. It can be made easy by the `cupy.get_array_module()` function. This function returns the numpy or cupy module based on arguments. A CPU/GPU generic function is defined using it like follows:



In [ ]:
# Stable implementation of log(1 + exp(x))
def softplus(x):
  xp = cp.get_array_module(x)
  return xp.maximum(0, x) + xp.log1p(xp.exp(-abs(x)))

Sometimes, an explicit conversion to a host or device array may be required. `cupy.asarray()` and `cupy.asnumpy()` can be used in agnostic implementations to get host or device arrays from either CuPy or NumPy arrays.

In [ ]:
y_cpu = np.array([4, 5, 6])
x_cpu + y_cpu
x_gpu + y_cpu

In [ ]:
cp.asnumpy(x_gpu) + y_cpu
cp.asnumpy(x_gpu) + cp.asnumpy(y_cpu)
x_gpu + cp.asarray(y_cpu)
cp.asarray(x_gpu) + cp.asarray(y_cpu)

## Cupy demo

* CuPy implements the multi-dimensional array of numpy on CUDA.
* CuPy has a large community of developers (on github),
under the direction by the company Preferred Networks.
* CuPy uses on-the-fly kernel synthesis:
for a required kernel call, it compiles the code of the kernel,
optimizes for shapes and dtypes of the arguments;
sends the compiled code to the GPU device; and
executes the kernel.
* The kernel code is cached, so the second call executes faster.

In [ ]:
### Numpy and CPU
s = time.time()
x_cpu = np.ones((1000,1000,1000))
e = time.time()
print(f"CPU: {e-s}")

### CuPy and GPU
s = time.time()
x_gpu = cp.ones((1000,1000,1000))
cp.cuda.Stream.null.synchronize()
e = time.time()
print(f"GPU: {e-s}")